# Trajectory replay

## Initial setup

In [2]:
import sys
sys.path.insert(0, "../python/")
from memory.ReplayMemory import ReplayMemory
import numpy as np
import random
import copy

## Numpy indexing

### Batch slicing

In [19]:
idx = np.random.randint(0, 100, 7)
print("idx:", idx)

x, y = np.meshgrid(idx, np.arange(5))
slices = np.transpose(x + y).flatten()
print("Slices:\n", slices)
slices %= 100
print("Corrected slices:\n", slices)

s1_0 = np.random.rand(100, 12, 84, 84) # mimics memory with capacity 100
s1_1 = np.random.rand(2)
s1 = [s1_0, s1_1]
print("Original:")
print("s1[0].shape:", s1[0].shape)
print("s1[0][slices][0, ...]:")
print(s1[0][slices][0, 0, :3, :3])
print("s1[0][slices][5, ...]:")
print(s1[0][slices][5, 0, :3, :3])

rs = np.reshape(s1[0][slices], [7, 5, 12, 84, 84]) # [num of traj, traj len, state_shape...]
print("Reshape:")
print("s1[0][slices].shape:", rs.shape)
print("s1[0][slices][0][...]:")
print(rs[0, 0, 0, :3, :3])
print("s1[0][slices][1][...]:")
print(rs[1, 0, 0, :3, :3])

idx: [58  8 24 44 72 70 19]
Slices:
 [58 59 60 61 62  8  9 10 11 12 24 25 26 27 28 44 45 46 47 48 72 73 74 75 76
 70 71 72 73 74 19 20 21 22 23]
Corrected slices:
 [58 59 60 61 62  8  9 10 11 12 24 25 26 27 28 44 45 46 47 48 72 73 74 75 76
 70 71 72 73 74 19 20 21 22 23]
Original:
s1[0].shape: (100, 12, 84, 84)
s1[0][slices][0, ...]:
[[ 0.53546728  0.34858874  0.99254931]
 [ 0.98171926  0.22050952  0.32387461]
 [ 0.50426754  0.38082718  0.53164911]]
s1[0][slices][5, ...]:
[[ 0.90396477  0.21205504  0.47445172]
 [ 0.63509333  0.78279289  0.96300232]
 [ 0.17780579  0.22307001  0.1404238 ]]
Reshape:
s1[0][slices].shape: (7, 5, 12, 84, 84)
s1[0][slices][0][...]:
[[ 0.53546728  0.34858874  0.99254931]
 [ 0.98171926  0.22050952  0.32387461]
 [ 0.50426754  0.38082718  0.53164911]]
s1[0][slices][1][...]:
[[ 0.90396477  0.21205504  0.47445172]
 [ 0.63509333  0.78279289  0.96300232]
 [ 0.17780579  0.22307001  0.1404238 ]]


### Reverse order
Applicable for class without reshaping

In [57]:
a = np.random.rand(30)
print("a:")
print(a)
print("a[[lists]]:")
print(a[[4, 9, 14, 19, 24, 29]])
print("a[fancy indexing]:")
print(a[4::5])
print("Reverse order:")
for i in range(5)[::-1]: 
    print(a[i::5])

a:
[ 0.08736947  0.26904687  0.18850901  0.15742123  0.14138586  0.10484942
  0.29992704  0.22221412  0.57461042  0.80538772  0.00261921  0.22227756
  0.20479822  0.9082098   0.57885134  0.35208548  0.84299166  0.9774215
  0.64349928  0.30795383  0.56341805  0.89320403  0.51121639  0.31188183
  0.72205378  0.2704125   0.41581256  0.55601417  0.59611474  0.86199004]
a[[lists]]:
[ 0.14138586  0.80538772  0.57885134  0.30795383  0.72205378  0.86199004]
a[fancy indexing]:
[ 0.14138586  0.80538772  0.57885134  0.30795383  0.72205378  0.86199004]
Reverse order:
[ 0.14138586  0.80538772  0.57885134  0.30795383  0.72205378  0.86199004]
[ 0.15742123  0.57461042  0.9082098   0.64349928  0.31188183  0.59611474]
[ 0.18850901  0.22221412  0.20479822  0.9774215   0.51121639  0.55601417]
[ 0.26904687  0.29992704  0.22227756  0.84299166  0.89320403  0.41581256]
[ 0.08736947  0.10484942  0.00261921  0.35208548  0.56341805  0.2704125 ]


## Class definition

### Reshaping into [# of trajectories, trajectory length, ...]
I like the way this is organized and that the Replay Memory class is doing some of the dirty work. However, this is incompatible with other forms of replay memory, in which the shape is [batch_size, ...].

In [61]:
class TrajectoryReplayMemory(ReplayMemory):
    def __init__(self, capacity, state_shape, num_game_var, input_overlap=0, 
                 trajectory_length=5):
        # Initialize base replay memory
        ReplayMemory.__init__(self, capacity, state_shape, num_game_var, input_overlap)

        # Initialize trajectory parameters
        self.tr_len = trajectory_length
        
    def get_sample(self, sample_size):
        # Get random minibatch of indices
        idx = np.random.randint(0, self.size, sample_size)
        x, y = np.meshgrid(idx, np.arange(self.tr_len))
        idx = np.transpose(x + y).flatten() # [i, i+1, ..., i+n, j, j+1, ..., j+n, k...]
        idx %= self.capacity # wrap end cases
        t = self.tr_len
        # TODO: find isterminal in sequences and cut short
        
        # s = [screen[trajectory id, trajectory step, state_shape...], gv[traj id, traj step, num_gv]]
        
        # Make list of states
        s1_sample, s2_sample = [], []

        # Get screen component
        s1_slice = self.s1[0][idx]
        if self.overlap > 0:
            # Stack overlapping frames from s1 to stored frames of s2 to
            # recreate full s2 state
            s2_slice = np.concatenate((self.s1[0][[idx] + [slice(None)] * self.chdim 
                                             + [slice(None, self.overlap)]], 
                                             self.s2[0][idx]), 
                                            axis=self.chdim+1)
        else:
            s2_slice = self.s2[0][idx]
        s1_sample.append(np.reshape(s1_slice, [sample_size, t] + self.state_shape))
        s2_sample.append(np.reshape(s2_slice, [sample_size, t] + self.state_shape))

        # Get game variable component
        s1_sample.append(np.reshape(self.s1[1][idx], [sample_size, t] + [self.num_game_var]))
        s2_sample.append(np.reshape(self.s2[1][idx], [sample_size, t] + [self.num_game_var]))

        # Get other transition parameters
        a_sample = np.reshape(self.a[idx], [sample_size, t])
        isterminal_sample = np.reshape(self.isterminal[idx], [sample_size, t])
        r_sample = np.reshape(self.r[idx], [sample_size, t])

        # Return importance sampling weights of one (stochastic distribution)
        w = np.ones([sample_size, t])

        return s1_sample, a_sample, s2_sample, isterminal_sample, r_sample, w, idx

In [62]:
capacity = 100
state_shape = [12, 84, 84]
num_game_var = 2
input_overlap = 3
trajectory_length = 10
memory = TrajectoryReplayMemory(capacity=capacity,
                                state_shape=state_shape,
                                num_game_var=num_game_var,
                                input_overlap=input_overlap,
                                trajectory_length=trajectory_length)
terminal_states = random.sample(range(capacity), 7)
s1, s2 = [], []
s2.append(np.random.rand(state_shape[0], state_shape[1], state_shape[2]))
s2.append([random.random()] * num_game_var)
for i in range(capacity):
    s1 = [s2[0], s2[1]]
    s2[0] = np.delete(s2[0], np.s_[0:input_overlap], axis=0)
    s2[0] = np.append(s2[0], np.random.rand(input_overlap, state_shape[1], state_shape[2]), axis=0)
    s2[1] = [random.random()] * num_game_var
    a = random.sample(range(4), 1)[0]
    r = random.random()
    isterminal = i in terminal_states
    memory.add_transition(s1, a, s2, isterminal, r)  

In [63]:
s1, a, s2, isterminal, r, w, idx = memory.get_sample(7)
print("idx:", idx)
print("s1[0].shape:", s1[0].shape)
print("s1[1].shape:", s1[1].shape)
print("a.shape:", a.shape)
print("s2[0].shape:", s2[0].shape)
print("s2[1].shape:", s2[1].shape)
print("isterminal.shape:", isterminal.shape)
print("r.shape:", r.shape)
print("w.shape:", w.shape)

idx: [ 1  2  3  4  5  6  7  8  9 10 64 65 66 67 68 69 70 71 72 73 95 96 97 98 99
  0  1  2  3  4 77 78 79 80 81 82 83 84 85 86 26 27 28 29 30 31 32 33 34 35
 87 88 89 90 91 92 93 94 95 96  4  5  6  7  8  9 10 11 12 13]
s1[0].shape: (7, 10, 12, 84, 84)
s1[1].shape: (7, 10, 2)
a.shape: (7, 10)
s2[0].shape: (7, 10, 12, 84, 84)
s2[1].shape: (7, 10, 2)
isterminal.shape: (7, 10)
r.shape: (7, 10)
w.shape: (7, 10)


### No reshaping, [batch_size, ...]
This returns experiences in the form [batch_size, ...], where batch_size = (# of trajectories, i.e. sample_size) * (trajectory length). This form is compatible with current Replay Memory classes.

In [14]:
class TrajectoryReplayMemory(ReplayMemory):
    def __init__(self, capacity, state_shape, num_game_var, input_overlap=0, 
                 trajectory_length=5):
        # Initialize base replay memory
        ReplayMemory.__init__(self, capacity, state_shape, num_game_var, input_overlap)

        # Initialize trajectory parameters
        self.tr_len = trajectory_length
        
    def get_sample(self, sample_size):
        # Get random minibatch of indices
        idx = np.random.randint(0, self.size, sample_size)
        x, y = np.meshgrid(idx, np.arange(self.tr_len))
        idx = np.transpose(x + y).flatten() # [i, i+1, ..., i+n, j, j+1, ..., j+n, k...]
        idx %= self.capacity # wrap end cases
        t = self.tr_len
        # TODO: find isterminal in sequences and cut short
          
        # Make list of states
        s1_sample, s2_sample = [], []

        # Get screen component
        s1_slice = self.s1[0][idx]
        if self.overlap > 0:
            # Stack overlapping frames from s1 to stored frames of s2 to
            # recreate full s2 state
            s2_slice = np.concatenate((self.s1[0][[idx] + [slice(None)] * self.chdim 
                                             + [slice(None, self.overlap)]], 
                                             self.s2[0][idx]), 
                                            axis=self.chdim+1)
        else:
            s2_slice = self.s2[0][idx]
        s1_sample.append(s1_slice)
        s2_sample.append(s2_slice)

        # Get game variable component
        s1_sample.append(self.s1[1][idx])
        s2_sample.append(self.s2[1][idx])

        # Get other transition parameters
        a_sample = self.a[idx]
        isterminal_sample = self.isterminal[idx]
        r_sample = self.r[idx]

        # Return importance sampling weights of one (stochastic distribution)
        w = np.ones([sample_size * t])

        return s1_sample, a_sample, s2_sample, isterminal_sample, r_sample, w, idx

In [15]:
capacity = 100
state_shape = [12, 84, 84]
num_game_var = 2
input_overlap = 3
trajectory_length = 10
memory = TrajectoryReplayMemory(capacity=capacity,
                                state_shape=state_shape,
                                num_game_var=num_game_var,
                                input_overlap=input_overlap,
                                trajectory_length=trajectory_length)
terminal_states = random.sample(range(capacity), 7)
s1, s2 = [], []
s2.append(np.random.rand(state_shape[0], state_shape[1], state_shape[2]))
s2.append([random.random()] * num_game_var)
for i in range(capacity):
    s1 = [s2[0], s2[1]]
    s2[0] = np.delete(s2[0], np.s_[0:input_overlap], axis=0)
    s2[0] = np.append(s2[0], np.random.rand(input_overlap, state_shape[1], state_shape[2]), axis=0)
    s2[1] = [random.random()] * num_game_var
    a = random.sample(range(4), 1)[0]
    r = random.random()
    isterminal = i in terminal_states
    memory.add_transition(s1, a, s2, isterminal, r)  

In [16]:
s1, a, s2, isterminal, r, w, idx = memory.get_sample(7)
print("idx:", idx)
print("s1[0].shape:", s1[0].shape)
print("s1[1].shape:", s1[1].shape)
print("a.shape:", a.shape)
print("s2[0].shape:", s2[0].shape)
print("s2[1].shape:", s2[1].shape)
print("isterminal.shape:", isterminal.shape)
print("r.shape:", r.shape)
print("w.shape:", w.shape)

idx: [46 47 48 49 50 51 52 53 54 55 78 79 80 81 82 83 84 85 86 87 42 43 44 45 46
 47 48 49 50 51 77 78 79 80 81 82 83 84 85 86 84 85 86 87 88 89 90 91 92 93
 90 91 92 93 94 95 96 97 98 99 24 25 26 27 28 29 30 31 32 33]
s1[0].shape: (70, 12, 84, 84)
s1[1].shape: (70, 2)
a.shape: (70,)
s2[0].shape: (70, 12, 84, 84)
s2[1].shape: (70, 2)
isterminal.shape: (70,)
r.shape: (70,)
w.shape: (70,)


In [24]:
sample = [s1[0][6::10], s1[1][6::10]]
print(len(sample), sample[0].shape, sample[1].shape)

2 (7, 12, 84, 84) (7, 2)


## Agent testing

In [3]:
# If using one or multiple GPUs
import os
os.environ["CUDA_VISIBLE_DEVICES"]=""

In [4]:
from helper import create_agent
import tensorflow as tf
from vizdoom import *
%load_ext autoreload
%autoreload 2

In [5]:
# Initializes DoomGame from config file
def initialize_vizdoom(config_file):
    print("Initializing doom... ", end=""), sys.stdout.flush()
    game = DoomGame()
    game.load_config(config_file)
    game.set_window_visible(True)
    game.init()
    print("Done.")
    return game

In [33]:
tf.reset_default_graph()
agent_file_path = "./trajectory_experience_replay/agent.json"
config_file_path = "../config/open_field.cfg"
results_dir = "./trajectory_experience_replay/results"

game = initialize_vizdoom(config_file_path)
agent = create_agent(agent_file_path,
                     game=game, 
                     params_file=None,
                     action_set=None,
                     output_directory=results_dir,
                     train_mode=True)

agent.initialize_new_episode()
for i in range(500): # memory capacity set to 100
    agent.perform_learning_step(1, 100)
    if game.is_episode_finished():
        agent.initialize_new_episode()
game.close()
print("Done.")

Initializing doom... Done.

Mapping of agent states --> network states:
screen --> Tensor("main_network/screen:0", shape=(?, 84, 84, 3), dtype=float32)
[GameVariable.HEALTH, GameVariable.VELOCITY_X, GameVariable.VELOCITY_Y] --> Tensor("main_network/game_vars:0", shape=(?, 3), dtype=float32)
Done.


In [32]:
game.close()